# 🔄 Convert Excel Supervisor Data to YAML

This notebook helps convert raw data exported from Microsoft Forms (as Excel) into a clean YAML file used by the [Supervisor Portfolio GitHub repository](https://github.com/AAUGS-DP-Biosciences-and-Drug-Research/supervisor-portfolio).

## ✨ Features
- Removes extra characters (e.g. newlines, non-breaking spaces)
- Converts structured multi-line text into YAML lists
- Outputs `supervisors.yaml` ready for upload to GitHub

## 📥 Input
Upload the `.xlsx` file exported from the Microsoft Form.

## 📤 Output
A clean `supervisors.yaml` file, downloadable from the Colab environment.

---

Once generated, place the YAML file into the `/data/` folder in your GitHub repo.

> 🧠 This makes updating and maintaining the portfolio easier and more transparent.


In [1]:
!pip install pyyaml pandas openpyxl
import re

def slugify(name):
    return re.sub(r'[^a-z0-9]+', '-', name.lower()).strip('-')


In [5]:
from google.colab import files
import io
import pandas as pd

print("⬆️ Please upload your Excel (.xlsx) file exported from Microsoft Forms")
uploaded = files.upload()

# Read Excel into DataFrame
excel_file = next(iter(uploaded))
df = pd.read_excel(io.BytesIO(uploaded[excel_file]))

# Clean column headers
def clean_header(header):
    return (
        header.replace('\n', ' ')
              .replace('\r', ' ')
              .replace('\xa0', ' ')
              .strip()
    )

df.columns = [clean_header(col) for col in df.columns]

print("✅ Cleaned headers:")
print(df.columns.tolist())

print("\n✅ Preview of the data:")
df.head()


⬆️ Please upload your Excel (.xlsx) file exported from Microsoft Forms


Saving Supervisor of the Doctoral Programme in Biosciences and Drug Research 1.xlsx to Supervisor of the Doctoral Programme in Biosciences and Drug Research 1 (2).xlsx
✅ Cleaned headers:
['Id', 'Start time', 'Completion time', 'Email', 'Name', 'Research group name', 'PI  name', 'Subject', 'Research group website', 'Link to AboCRIS profile', 'Areas of Expertise', 'Research projects', 'Special methodologies & techniques', 'Major funding source(s) and international network(s)', 'Five selected publications', 'Upload a profile photo', 'I give the permission to post my picture online']

✅ Preview of the data:


,Id,Start time,Completion time,Email,Name,Research group name,PI name,Subject,Research group website,Link to AboCRIS profile,Areas of Expertise,Research projects,Special methodologies & techniques,Major funding source(s) and international network(s),Five selected publications,Upload a profile photo,I give the permission to post my picture online
0,2,2025-04-26 18:49:40,2025-04-26 18:51:50,guillaume.jacquemet@abo.fi,Guillaume Jacquemet,Cell Migration Lab,Guillaume Jacquemet,Cell Biology,https://cellmig.org/,https://research.abo.fi/en/persons/guillaume-j...,Cancer cell biology\nCell adhesion and migrati...,The role of filopodia during breast cancer pro...,"Microscopy (super-resolution microscopy, live ...",Sigrid Juselius Foundation\nWellcome Trust\nAc...,TLNRD1 is a CCM complex component and regulate...,https://abofi-my.sharepoint.com/personal/guill...,Yes


In [7]:
import yaml
import re

def slugify(name):
    return re.sub(r'[^a-z0-9]+', '-', name.lower()).strip('-')

def clean_field(value):
    return str(value).replace('\xa0', ' ').replace('\r', ' ').replace('\n', ' ').strip()

def safe_splitlines(value):
    return [line.strip() for line in str(value).splitlines() if line.strip()]

supervisors = []

for _, row in df.iterrows():
    name = clean_field(row.get("Name", ""))
    if not name:
        continue

    slug = slugify(name)

    photo_raw = clean_field(row.get("Upload a profile photo", ""))
    photo_ext = "." + photo_raw.split(".")[-1] if "." in photo_raw else ".jpg"
    photo_filename = f"{slug}{photo_ext}"

    supervisor = {
        "name": name,
        "slug": slug,
        "group": clean_field(row.get("Research group name", "")),
        "unit": clean_field(row.get("Subject", "")),
        "university": "Åbo Akademi University",
        "lab_website": clean_field(row.get("Research group website", "")),
        "cris_profile": clean_field(row.get("Link to AboCRIS profile", "") or row.get("Link to AboCRIS profile", "")),
        "photo": photo_filename,
        "expertise": safe_splitlines(row.get("Areas of Expertise", "")),
        "projects": safe_splitlines(row.get("Research projects", "")),
        "techniques": safe_splitlines(row.get("Special methodologies & techniques", "")),
        "funding": safe_splitlines(row.get("Major funding source(s) and international network(s)", "")),
        "publications": safe_splitlines(row.get("Five selected publications", "")),
        "keywords": clean_field(row.get("Key words", "")),
    }

    supervisors.append(supervisor)

yaml_string = yaml.dump(
    supervisors,
    allow_unicode=True,
    sort_keys=False,
    default_flow_style=False
)

print("✅ YAML preview:")
print(yaml_string[:2000] + "\n...")


✅ YAML preview:
- name: Guillaume Jacquemet
  slug: guillaume-jacquemet
  group: Cell Migration Lab
  unit: Cell Biology
  university: Åbo Akademi University
  lab_website: https://cellmig.org/
  cris_profile: https://research.abo.fi/en/persons/guillaume-jacquemet
  photo: guillaume-jacquemet.jpg
  expertise:
  - Cancer cell biology
  - Cell adhesion and migration
  - Microscopy and live imaging
  - Image analysis
  projects:
  - The role of filopodia during breast cancer progression
  - Cancer cell communication via filopodia trans- endocytosis
  - Deciphering the mechanisms of pancreatic cancer metastasis
  - The role of mechanosensitive calcium channels in melanoma
  - The role of TLNRD1 in endothelium homeostasis
  - Democratising deep learning for microscopy with ZeroCostDL4Mic
  techniques:
  - Microscopy (super-resolution microscopy, live imaging, traction force microscopy)
  - Image analysis, deep learning and computer vision
  - Cell and molecular biology
  - Mass spectrometry

In [8]:
with open("supervisors.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_string)

files.download("supervisors.yaml")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>